In [1]:
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, ParameterGrid
import torch
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, default_data_collator, DebertaV2Tokenizer, PegasusForConditionalGeneration, PegasusTokenizer
from tqdm.notebook import tqdm

paths = ["data/mdbrt_xnl_train.csv", "data/rbrt_3_train.csv"]

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
class Model:
    def __init__(self):
        pass
        
    def create_datasets(self, path):
        column = "Группа тем"
        data = pd.read_csv(path)
        
        train, test = train_test_split(data,
                                       test_size=0.33, random_state=42, stratify=data[column])
        
        train, test = train.reset_index(drop=True), test.reset_index(drop=True)

        return train, test

In [3]:
#data = pd.read_csv("data/rbrt_3_train.csv")

In [4]:
param_grid = {
    'iterations': [1000, 1200]
}
grid = list(ParameterGrid(param_grid))

In [5]:
mlflow.set_experiment('few_shot')

<Experiment: artifact_location='file:///workspace/mlruns/383196265594095667', creation_time=1700839358903, experiment_id='383196265594095667', last_update_time=1700839358903, lifecycle_stage='active', name='few_shot', tags={}>

In [6]:
for path in paths:

    train_, test_ = Model().create_datasets(path) 
    
    for params in grid:
        try:
            with mlflow.start_run(nested=True):
                catboost_model = CatBoostClassifier(**params, verbose=0, random_seed=42)
                catboost_model.fit(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]), train_["Группа тем"])
    
                predictions = catboost_model.predict(test_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]))
                
                accuracy = accuracy_score(test_["Группа тем"], predictions)
                f1 = f1_score(test_["Группа тем"], predictions, average='weighted') 
                report = classification_report(test_["Группа тем"], predictions, output_dict=True)
                report_text =  classification_report(test_["Группа тем"], predictions)
    
                
                mlflow.log_metric("report_accuracy", report['accuracy'])
                mlflow.log_metric("macro avg_precision", report['macro avg']['precision'])
                mlflow.log_metric("macro avg_recall", report['macro avg']['recall'])
                mlflow.log_metric("macro avg_f1-score", report['macro avg']['f1-score'])
                mlflow.log_metric("weighted avg_precision", report['weighted avg']['precision'])
                mlflow.log_metric("weighted avg_recall", report['weighted avg']['recall'])
                mlflow.log_metric("weighted avg_f1-score", report['weighted avg']['f1-score'])
    
                mlflow.log_text(report_text, "classification_report.txt")
                
                mlflow.set_tag("embedding_name", path.lstrip("data/"))
                mlflow.set_tag("dataset_name", "clear_v2")
                mlflow.set_tag("model_name", "catboost")
                mlflow.catboost.log_model(catboost_model, "model")
                mlflow.log_params(params)
                mlflow.log_metrics({'accuracy': accuracy, 'f1-weighted': f1})
        except Exception as e:
            error_name = type(e).__name__
            print(f"Caught an error: {error_name}")

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif